# 设置数据文件存放根目录（不直接放在该项目中，该项目有github仓库，避免大内存数据影响代码推送）

In [1]:
boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"

# 建立映射表

In [2]:
import pandas as pd
import os

# ========================
# 路径配置
# ========================
# boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"
raw_csv_dir = os.path.join(boot_file_adr, "raw_csv")
save_dir = os.path.join(boot_file_adr, "mapping_table")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "KAIST_vibration_mapping.csv")

# ========================
# 获取文件列表
# ========================
csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])

# ========================
# 标签映射（Normal=1，不同转速/故障独立编号）
# ========================
mapping_rules = {
    "Normal": 1,
    "BPFI_03": 2,
    "BPFI_10": 3,
    "BPFI_30": 4,
    "BPFO_03": 5,
    "BPFO_10": 6,
    "BPFO_30": 7,
    "Misalign_01": 8,
    "Misalign_03": 9,
    "Misalign_05": 10,
    "Unbalance_0583mg": 11,
    "Unbalance_1169mg": 12,
    "Unbalance_1751mg": 13,
    "Unbalance_2239mg": 14,
    "Unbalance_3318mg": 15,
}

# ========================
# 生成映射表
# ========================
records = []
for f in csv_files:
    matched_label = None
    for key, label in mapping_rules.items():
        if key in f:
            matched_label = label
            break
    if matched_label is None:
        matched_label = -1
    records.append({
        "filename": f,
        "label": matched_label
    })

df = pd.DataFrame(records)
df.to_csv(save_path, index=False)

print(f"✅ 已生成映射表：{save_path}")
display(df)


✅ 已生成映射表：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv


,filename,label
0,0Nm_BPFI_03.csv,2
1,0Nm_BPFI_10.csv,3
2,0Nm_BPFI_30.csv,4
3,0Nm_BPFO_03.csv,5
4,0Nm_BPFO_10.csv,6
5,0Nm_BPFO_30.csv,7
6,0Nm_Misalign_01.csv,8
7,0Nm_Misalign_03.csv,9
8,0Nm_Misalign_05.csv,10
9,0Nm_Normal.csv,1


# 数据混合

## 全类别混合，用于基本模型训练评估

### mixed data->csv

In [11]:
import os
import pandas as pd

# ===================== 1️⃣ 参数设置 =====================
raw_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/raw_csv"
mapping_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
used_index_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/used_indices.csv"

# ========== 你需要的每类抽取数量 ==========
sample_dict = {
    1: 4000,
    2: 2000,
    3: 3000,
    4: 1000,
    5: 1000,
    6: 2000,
    7: 3000,
    8: 1000,
    9: 1500,
    10: 700,
    11: 500,
    12: 1000,
    13: 1000,
    14: 1000,
}

# ===================== 2️⃣ 读取映射表 =====================
mapping = pd.read_csv(mapping_path)
print(f"📘 映射表已读取，共 {len(mapping)} 条记录")

# ===================== 3️⃣ 加载已使用索引 =====================
if os.path.exists(used_index_path):
    used_indices = pd.read_csv(used_index_path)
else:
    used_indices = pd.DataFrame(columns=["source_file", "index", "label"])

# ===================== 4️⃣ 初始化输出文件 =====================
if os.path.exists(save_path):
    os.remove(save_path)
    print("⚙️ 已清空旧的 mixed_allclass.csv")
first_write = True

# ===================== 5️⃣ 实际抽样开始 =====================
# 遍历每一类
for label, target_n in sample_dict.items():

    print(f"\n==============================")
    print(f"🎯 开始处理类别 {label}，目标抽取 {target_n} 条")
    print(f"==============================")

    # 统计当前类别已抽取多少
    already_used = used_indices.loc[used_indices["label"] == label].shape[0]
    print(f"📊 当前类别 {label} 已有 {already_used} 条")

    if already_used >= target_n:
        print(f"✅ 类别 {label} 样本已足够（{already_used} >= {target_n}），跳过本类。")
        continue

    # 还需要抽的数
    remain = target_n - already_used
    print(f"🔍 还需 {remain} 条样本")

    # 遍历所有文件，属于该类的都尝试抽取
    for fname in os.listdir(raw_dir):

        if not fname.endswith(".csv"):
            continue
        if "2Nm" in fname:   # 排除 2Nm 文件
            continue

        # 找当前文件的标签
        entry = mapping.loc[mapping["filename"] == fname]
        if entry.empty:
            continue

        file_label = int(entry["label"].values[0])
        if file_label != label:
            continue

        file_path = os.path.join(raw_dir, fname)
        df = pd.read_csv(file_path)
        df["label"] = label
        df["source_file"] = fname

        # 当前文件未使用的样本
        used_idx_this_file = used_indices.loc[used_indices["source_file"] == fname, "index"].tolist()
        df_avail = df.loc[~df.index.isin(used_idx_this_file)]

        if df_avail.empty:
            print(f"⚠️ {fname} 可用数据为 0，跳过。")
            continue

        # 本文件可抽取数量
        take_n = min(remain, len(df_avail))

        sampled_df = df_avail.sample(n=take_n, random_state=42)

        # 写入输出文件
        sampled_df.to_csv(save_path, mode="a", header=first_write, index=False)
        first_write = False

        # 更新 used_indices
        new_used = pd.DataFrame({
            "source_file": fname,
            "index": sampled_df.index,
            "label": label
        })
        used_indices = pd.concat([used_indices, new_used], ignore_index=True)

        print(f"📦 从 {fname} 抽取 {take_n} 条。")

        remain -= take_n
        if remain <= 0:
            print(f"🎉 类别 {label} 已抽满 {target_n} 条。")
            break

# ===================== 6️⃣ 保存使用记录 =====================
used_indices.to_csv(used_index_path, index=False)
print("\n==============================")
print(f"🏁 所有类别抽样完成！")
print(f"📁 已保存：{save_path}")
print(f"📁 使用记录：{used_index_path}")
print("==============================")


📘 映射表已读取，共 15 条记录
⚙️ 已清空旧的 mixed_allclass.csv

🎯 开始处理类别 1，目标抽取 4000 条
📊 当前类别 1 已有 0 条
🔍 还需 4000 条样本
📦 从 0Nm_Normal.csv 抽取 4000 条。
🎉 类别 1 已抽满 4000 条。

🎯 开始处理类别 2，目标抽取 2000 条
📊 当前类别 2 已有 0 条
🔍 还需 2000 条样本
📦 从 0Nm_BPFI_03.csv 抽取 2000 条。
🎉 类别 2 已抽满 2000 条。

🎯 开始处理类别 3，目标抽取 3000 条
📊 当前类别 3 已有 0 条
🔍 还需 3000 条样本
📦 从 0Nm_BPFI_10.csv 抽取 3000 条。
🎉 类别 3 已抽满 3000 条。

🎯 开始处理类别 4，目标抽取 1000 条
📊 当前类别 4 已有 0 条
🔍 还需 1000 条样本
📦 从 0Nm_BPFI_30.csv 抽取 1000 条。
🎉 类别 4 已抽满 1000 条。

🎯 开始处理类别 5，目标抽取 1000 条
📊 当前类别 5 已有 0 条
🔍 还需 1000 条样本
📦 从 0Nm_BPFO_03.csv 抽取 1000 条。
🎉 类别 5 已抽满 1000 条。

🎯 开始处理类别 6，目标抽取 2000 条
📊 当前类别 6 已有 0 条
🔍 还需 2000 条样本
📦 从 0Nm_BPFO_10.csv 抽取 2000 条。
🎉 类别 6 已抽满 2000 条。

🎯 开始处理类别 7，目标抽取 3000 条
📊 当前类别 7 已有 0 条
🔍 还需 3000 条样本
📦 从 0Nm_BPFO_30.csv 抽取 3000 条。
🎉 类别 7 已抽满 3000 条。

🎯 开始处理类别 8，目标抽取 1000 条
📊 当前类别 8 已有 0 条
🔍 还需 1000 条样本
📦 从 0Nm_Misalign_01.csv 抽取 1000 条。
🎉 类别 8 已抽满 1000 条。

🎯 开始处理类别 9，目标抽取 1500 条
📊 当前类别 9 已有 0 条
🔍 还需 1500 条样本
📦 从 0Nm_Misalign_03.csv 抽取 1500 条。
🎉 类别 9 已抽满 1500 条。

🎯 开始处理类别 1

### mixed_data->pyg

#### 时间邻点建图

In [12]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph
import pandas as pd
import os

# ================== 2️⃣ 参数设置 ==================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time"
num_edges = 10  # 每个节点的边数（上下平均分配）

# ================== 3️⃣ 预处理：忽略最后一列 ==================
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ 找不到混合文件: {csv_path}")

df = pd.read_csv(csv_path)
print(f"📊 原始数据形状: {df.shape}")

# 去掉最后一列（通常为标签）
df = df.iloc[:, :-1]
print(f"🧹 已移除最后一列后形状: {df.shape}")

# 保存临时文件供建图使用
temp_csv_path = csv_path.replace(".csv", "_nofinal.csv")
df.to_csv(temp_csv_path, index=False)
print(f"✅ 已生成去除标签列的临时文件: {temp_csv_path}")

# ================== 4️⃣ 调用函数建图 ==================
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=temp_csv_path,
    save_dir=save_dir,
    num_edges=num_edges
)

# ================== 5️⃣ 输出结果路径 ==================
print("\n✅ 局部时序图已构建完成：")
print(f"📁 节点文件: {nodes_csv}")
print(f"📁 边文件:   {edges_csv}")
print(f"📁 图文件:   {graph_pt}")



✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📊 原始数据形状: (22700, 7)
🧹 已移除最后一列后形状: (22700, 6)
✅ 已生成去除标签列的临时文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv
✅ 图结构构建完成，共 22700 个节点，227000 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/graph.pt
🧩 特征维度: 4 (已自动忽略首列与标签列)

✅ 局部时序图已构建完成：
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/edges.csv
📁 图文件:   /home/charles/HZU/Data_processed/my

In [13]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import add_random_masks_with_label_split
import pandas as pd
import os
import torch

# ================== 2️⃣ 参数设置 ==================
ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/graph.pt"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/mixed_allclass_arabic_graph_withmask.pt"

# ================== 3️⃣ 调用函数生成掩码 ==================
print("🚀 开始为图添加掩码 ...")
new_graph_path = add_random_masks_with_label_split(
    graph_path=graph_path,
    save_path=save_path,      # 若想覆盖源文件则直接传 None
    ratios=ratios,
    train_label_ratio=0.4,    # 训练集中40%为有标签
)
print("\n✅ 掩码生成与保存完成。")

# ================== 4️⃣ 加载验证 ==================
print("\n🧩 验证生成的图结构与掩码 ...")
data = torch.load(new_graph_path)

# === 基本结构 ===
print(f"\n📦 图数据结构: {data}")
print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
print(f"前5个标签: {data.y[:5].tolist()}")

# === 掩码存在性检查 ===
mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
for attr in mask_attrs:
    exists = hasattr(data, attr)
    print(f"{attr:<25}: {'✅ 存在' if exists else '❌ 不存在'}")

# === 掩码分布验证 ===
if hasattr(data, "train_mask"):
    total_nodes = data.x.size(0)
    n_train = int(data.train_mask.sum())
    n_val = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
    n_test = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
    n_withlabel = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
    n_nolabel = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

    print("\n📊 掩码数量验证：")
    print(f"  train_mask         = {n_train}")
    print(f"  val_mask           = {n_val}")
    print(f"  test_mask          = {n_test}")
    print(f"  └─ train_withlabel = {n_withlabel}")
    print(f"  └─ train_nolabel   = {n_nolabel}")
    print(f"  合计节点数         = {total_nodes}")
    if n_train + n_val + n_test == total_nodes:
        print("✅ 掩码划分总数匹配节点总数")
    else:
        print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

print("\n🎯 掩码验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始为图添加掩码 ...
📦 已加载图文件，共 22700 个节点。
✅ 已生成主掩码：train=13620, val=4540, test=4540
🎯 训练集内部划分：with_label=5448, no_label=8172
💾 已将含掩码的新图文件保存至：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/mixed_allclass_arabic_graph_withmask.pt

✅ 掩码生成与保存完成。

🧩 验证生成的图结构与掩码 ...

📦 图数据结构: Data(x=[22700, 4], edge_index=[2, 454000], y=[22700], train_mask=[22700], val_mask=[22700], test_mask=[22700], train_withlabel_mask=[22700], train_nolabel_mask=[22700])
x shape: torch.Size([22700, 4]), y shape: torch.Size([22700])
前5个标签: [1, 1, 1, 1, 1]
train_mask               : ✅ 存在
val_mask                 : ✅ 存在
test_mask                : ✅ 存在
train_withlabel_mask     : ✅ 存在
train_nolabel_mask       : ✅ 存在

📊 掩码数量验证：
  train_mask         = 13620
  val_mask     

/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib/build_pyg_data.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

#### 余弦相似度建图

In [14]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_similarity_knn_graph
import pandas as pd
import torch
import os

# ================== 2️⃣ 参数设置 ==================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv"
save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn"

# ================== 3️⃣ 调用建图函数 ==================
print("🚀 开始构建基于余弦相似度的KNN图 ...")
nodes_csv, edges_csv, graph_pt = build_similarity_knn_graph(
    csv_path=csv_path,
    save_dir=save_dir,
    num_edges=10
)
print("\n✅ 图结构已生成：")
print(f"📁 节点文件: {nodes_csv}")
print(f"📁 边文件:   {edges_csv}")
print(f"📁 图文件:   {graph_pt}")

# ================== 4️⃣ 验证生成的图数据 ==================
print("\n🧩 验证生成的 PyG 图数据结构 ...")

# 读取 PyG 对象
data = torch.load(graph_pt, weights_only=False)

# === 基础信息 ===
num_nodes = data.x.size(0)
num_edges = data.edge_index.size(1)
num_features = data.x.size(1)
has_label = hasattr(data, "y") and data.y is not None

print(f"📦 Data 对象结构: {data}")
print(f"🧮 节点数: {num_nodes}")
print(f"🧮 边数: {num_edges // 2}（无向边）")
print(f"🧩 特征维度: {num_features}")
print(f"🏷️  标签存在: {'✅ 是' if has_label else '❌ 否'}")

# === 标签验证 ===
if has_label:
    y = data.y.cpu().numpy()
    unique_labels = sorted(set(y))
    print(f"🎯 标签类别数量: {len(unique_labels)} | 类别: {unique_labels[:10]}")
    print(f"前5个标签: {y[:5]}")
else:
    print("⚠️ 未检测到 data.y，标签列可能缺失或未正确指定。")

# === 节点/边文件验证 ===
print("\n📊 验证节点和边 CSV 文件 ...")
df_nodes = pd.read_csv(nodes_csv)
df_edges = pd.read_csv(edges_csv)

print(f"✅ 节点文件行数: {len(df_nodes)} | 与图节点数 {'一致 ✅' if len(df_nodes)==num_nodes else '不一致 ⚠️'}")
print(f"✅ 边文件行数: {len(df_edges)} | 每两行对应一条无向边")

if "source" in df_edges.columns and "target" in df_edges.columns:
    unique_nodes = pd.concat([df_edges["source"], df_edges["target"]]).nunique()
    print(f"🧮 边中涉及的唯一节点数: {unique_nodes}")
else:
    print("⚠️ 边文件缺少 'source' 或 'target' 列！")

print("\n🎯 图数据验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始构建基于余弦相似度的KNN图 ...
📊 已读取数据: (22700, 6)
🧩 使用特征列数: 4 | 节点数: 22700
⚙️ 分块计算余弦相似度，每块 5000 节点，目标邻点数=10
  ✅ 已处理 5000/22700 节点块
  ✅ 已处理 10000/22700 节点块
  ✅ 已处理 15000/22700 节点块
  ✅ 已处理 20000/22700 节点块
  ✅ 已处理 22700/22700 节点块

✅ 图构建完成，共 22700 个节点，227000 条无向边。
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_edges.csv
📁 图文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt

✅ 图结构已生成：
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_nodes.csv
📁 边文件:   /ho

In [15]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import add_random_masks_with_label_split
import pandas as pd
import torch
import os

# ================== 2️⃣ 参数设置 ==================
ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt"
save_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt"

# ================== 3️⃣ 生成掩码 ==================
print("🚀 开始为图添加随机掩码 ...")
new_graph_path = add_random_masks_with_label_split(
    graph_path=graph_path,
    save_path=save_path,     # 若想覆盖源文件则直接传 None
    ratios=ratios,
    train_label_ratio=0.4,   # 训练集中40%为有标签
)
print(f"\n✅ 掩码生成与保存完成：{new_graph_path}")

# ================== 4️⃣ 验证生成结果 ==================
print("\n🧩 验证生成的图结构与掩码 ...")

# 读取 PyG 数据
data = torch.load(new_graph_path, weights_only=False)

# === 基本信息 ===
print(f"\n📦 图数据结构: {data}")
print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
print(f"前5个标签: {data.y[:5].tolist()}")

# === 掩码存在性检测 ===
mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
print("\n📋 掩码存在性检查：")
for attr in mask_attrs:
    print(f"  {attr:<25}: {'✅ 存在' if hasattr(data, attr) else '❌ 不存在'}")

# === 掩码统计 ===
if hasattr(data, "train_mask"):
    total_nodes = data.x.size(0)
    n_train = int(data.train_mask.sum())
    n_val   = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
    n_test  = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
    n_with  = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
    n_nol   = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

    print("\n📊 掩码数量验证：")
    print(f"  train_mask         = {n_train}")
    print(f"  val_mask           = {n_val}")
    print(f"  test_mask          = {n_test}")
    print(f"  └─ train_withlabel = {n_with}")
    print(f"  └─ train_nolabel   = {n_nol}")
    print(f"  合计节点数         = {total_nodes}")
    if n_train + n_val + n_test == total_nodes:
        print("✅ 掩码划分总数匹配节点总数")
    else:
        print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

# === 标签统计 ===
unique_labels = sorted(set(data.y.cpu().numpy().tolist()))
print("\n🎯 标签分布检查：")
print(f"  标签类别数: {len(unique_labels)}")
print(f"  类别示例: {unique_labels[:10]}")

print("\n🎯 掩码与标签验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始为图添加随机掩码 ...
📦 已加载图文件，共 22700 个节点。
✅ 已生成主掩码：train=13620, val=4540, test=4540
🎯 训练集内部划分：with_label=5448, no_label=8172
💾 已将含掩码的新图文件保存至：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt

✅ 掩码生成与保存完成：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt

🧩 验证生成的图结构与掩码 ...

📦 图数据结构: Data(x=[22700, 4], edge_index=[2, 454000], y=[22700], train_mask=[22700], val_mask=[22700], test_mask=[22700], train_withlabel_mask=[22700], train_nolabel_mask=[22700])
x shape: torch.Size([22700, 4]), y shape: torch.Size([22700])
前5个标签: [1, 1, 1, 1, 1]

📋 掩码存在性检查：
  train_mask               : ✅ 存在
  val_mask                 : ✅ 存在
  test_mas

/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib/build_pyg_data.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

#### time + cos_knn 融合建图